In [1]:
import pandas as pd
from etl_resources import sqlite_connection
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandasql as psql
from sklearn.preprocessing import MinMaxScaler
pd.options.mode.chained_assignment = None 

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

2021-11-30 19:31:06.425648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-30 19:31:06.425714: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def ticker_list():
    
    con = sqlite_connection()
    cur = con.cursor()
    cur.execute('''
    select distinct w.ticker from weekly_prices_clean w
        inner join (select ticker from weekly_prices_clean
        group by ticker
        having max(date) > '2021-01-01') t on t.ticker=w.ticker
        
    ''')
    res = cur.fetchall()
    res = [val[0] for val in res]
    
    return res

In [4]:
def base_df(ticker):
    
    '''
    This function returns the base time series dataframe (date and close)
    '''
    
    con = sqlite_connection()
        
    df = pd.read_sql(f'''select date,close 
    from weekly_prices_clean where ticker='{ticker}' --and date>'2017-12-31' 
    group by date,close
    order by date asc''',con=con)
    
    df = df.filter(['close'])
        
    return df

In [5]:
def mean_abs_perc_err(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
class TFCallback(tf.keras.callbacks.Callback):
    
    def on_epoch_end(self, epochs, logs={}) :
        threshold = 0.002
        if(logs.get('loss') is not None and logs.get('loss') < threshold) :
            print('\nLoss dropped below {}, cancelling further training'.format(str(threshold)))
            self.model.stop_training = True


In [7]:
def build_lstm(x_train, y_train, x_test, y_test, scaler):
    
    
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    callbacks = TFCallback()
    
    # Train the model
    model.fit(x_train, y_train, batch_size=1, epochs=50, callbacks=[callbacks])
    
    # Convert the data to a numpy array
    x_test = np.array(x_test)

    # Reshape the data
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

    # Get the models predicted price values 
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    # Get the root mean squared error (RMSE)
    rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
    mape = mean_abs_perc_err(y_test, predictions)
    print('RMSE:',rmse)
    print('MAPE:',mape)
    
    return predictions
    

In [ ]:
def main():
    # following https://www.kaggle.com/faressayah/stock-market-analysis-prediction-using-lstm
    tickers = ticker_list()
    #metrics_list = list()
    
    for ticker in tickers:
        
        print(f"Building model for {ticker}")
        
        #metrics = dict()
        
        # Build the dataset and split
        df = base_df(ticker)
        dataset = df.values
        scaler = MinMaxScaler(feature_range=(0,1))
        scaled_data = scaler.fit_transform(dataset)
        training_data_len = int(np.ceil( len(dataset) * .95 ))
        train_data = scaled_data[0:int(training_data_len), :]
    
        # Split the data into x_train and y_train data sets
        x_train = []
        y_train = []
        #print([i for i in range(100, len(train_data))])
        for i in range(60, len(train_data)):
            x_train.append(train_data[i-60:i, 0])
            y_train.append(train_data[i, 0])
       
        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

        test_data = scaled_data[training_data_len - 60: , :]
        
        # Create the data sets x_test and y_test
        x_test = []
        y_test = dataset[training_data_len:, :]
        for i in range(60, len(test_data)):
            x_test.append(test_data[i-60:i, 0])
    
        # Convert the data to a numpy array
        x_test = np.array(x_test)

        # Reshape the data
        x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
        
        # Train & Test LSTM
        predictions = build_lstm(x_train, y_train, x_test, y_test, scaler)
        
        train = df[:training_data_len]
        valid = df[training_data_len:]
        valid['Predictions'] = predictions
        # Visualize the data
        plt.figure(figsize=(16,6))
        plt.title('Model')
        plt.xlabel('Date', fontsize=18)
        plt.ylabel('Close Price USD ($)', fontsize=18)
        plt.plot(train['close'])
        plt.plot(valid[['close', 'Predictions']])
        plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
        plt.savefig(f'../data/visualization/lstm/{ticker}.png', facecolor='white', transparent=False)
        plt.close('all')
        

main()      

Building model for MMM


2021-11-30 19:31:08.116100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-30 19:31:08.116133: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-30 19:31:08.116151: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jameslaptop-Lenovo-Yoga-2-Pro): /proc/driver/nvidia/version does not exist
2021-11-30 19:31:08.116339: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-30 19:31:08.569299: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] N

Epoch 1/50
1032/1032 [==============================] - 31s 28ms/step - loss: 0.0030
Epoch 2/50
1032/1032 [==============================] - 32s 31ms/step - loss: 0.0012

Loss dropped below 0.002, cancelling further training
RMSE: 5.03580128552323
MAPE: 2.2554948232578567
Building model for BKNG
Epoch 1/50
1032/1032 [==============================] - 36s 33ms/step - loss: 0.0023
Epoch 2/50
1032/1032 [==============================] - 34s 33ms/step - loss: 0.0014

Loss dropped below 0.002, cancelling further training
RMSE: 196.59756307470087
MAPE: 7.68166606268815
Building model for ABT
Epoch 1/50
1032/1032 [==============================] - 33s 30ms/step - loss: 0.0023
Epoch 2/50
1032/1032 [==============================] - 32s 31ms/step - loss: 0.0010

Loss dropped below 0.002, cancelling further training
RMSE: 7.758655694677558
MAPE: 5.885838584861403
Building model for ABBV
Epoch 1/50
379/379 [==============================] - 14s 31ms/step - loss: 0.0112
Epoch 2/50
379/379 [=======

/tmp/ipykernel_61847/2964832072.py:53: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(16,6))


Epoch 1/50
528/528 [==============================] - 19s 31ms/step - loss: 0.0033
Epoch 2/50
528/528 [==============================] - 17s 32ms/step - loss: 0.0013

Loss dropped below 0.002, cancelling further training
RMSE: 43.91978731716733
MAPE: 4.622280107740823
Building model for CVX
Epoch 1/50
1032/1032 [==============================] - 33s 31ms/step - loss: 0.0080
Epoch 2/50
1032/1032 [==============================] - 33s 32ms/step - loss: 0.0036
Epoch 3/50
1032/1032 [==============================] - 33s 32ms/step - loss: 0.0028
Epoch 4/50
1032/1032 [==============================] - 33s 32ms/step - loss: 0.0026
Epoch 5/50
1032/1032 [==============================] - 33s 32ms/step - loss: 0.0026
Epoch 6/50
1032/1032 [==============================] - 39s 38ms/step - loss: 0.0026
Epoch 7/50
1032/1032 [==============================] - 36s 34ms/step - loss: 0.0024
Epoch 8/50
1032/1032 [==============================] - 35s 34ms/step - loss: 0.0022
Epoch 9/50
1032/1032 [======